In [6]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
import pandas as pd
import datetime
import pytz
import schedule
import time
from datetime import datetime

In [18]:
url = "https://www.zse.co.zw/price-sheet/"

In [19]:
def get_todays_data():
  response = requests.get(url)
  data = pd.read_html(response.text)
  df = pd.DataFrame(data[0])
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.dropna()
  return df

def get_open_price():
  df = get_todays_data()
  open_price = df[['Company Name','Opening Price']].T
  open_price.columns  = open_price.iloc[0]
  open_price = open_price[1:]
  open_price.index.name = 'Date'
  open_price = open_price.rename(index={open_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return open_price

def get_close_price():
  df = get_todays_data()
  close_price = df[['Company Name','Closing Price']].T
  close_price.columns  = close_price.iloc[0]
  close_price = close_price[1:]
  close_price.index.name = 'Date'
  close_price = close_price.rename(index={close_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return close_price

def get_vol_traded():
  df = get_todays_data()
  vol_traded = df[['Company Name','Total Traded Volume']].T
  vol_traded.columns  = vol_traded.iloc[0]
  vol_traded = vol_traded[1:]
  vol_traded.index.name = 'Date'
  vol_traded = vol_traded.rename(index={vol_traded.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return vol_traded

In [20]:
global open_price
global close_price
global vol_traded
open_price = get_open_price()
close_price = get_close_price()
vol_traded = get_vol_traded()

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


In [21]:
def update_data():
  global open_price
  open_price = pd.concat([open_price, get_open_price()], axis=0) # Update the variables
  global close_price
  close_price = pd.concat([close_price, get_close_price()], axis=0)
  global vol_traded
  vol_traded = pd.concat([vol_traded, get_vol_traded()], axis=0)

  return print(open_price), print(close_price), print(vol_traded)



In [26]:
open_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-15 13:37:04.878782+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:38:50.545881+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:39:23.879831+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:40:43.648451+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:41:03.965778+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:41:37.997257+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:43:17.261272+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:43:52.866361+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38
2024-10-15 13:45:31.056505+02:00,700,55.25,8,1.5,2.64,7199.9,1400,1800,281.5,230.0009,...,2.1058,182.4,280,6.3123,250,4.4249,814.5714,0.0018,6.2914,38


In [27]:
close_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-15 13:37:09.968327+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:38:54.181915+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:39:28.333809+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:40:47.243273+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:41:08.295026+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:41:44.388653+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:43:21.265972+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:43:57.036178+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38
2024-10-15 13:45:35.117737+02:00,700,55.25,7.95,1.5,2.64,7199.95,1400,1800,281.5,240,...,2.1,181.5726,280,6,250,4.5,814.5714,0.0018,6,38


In [28]:
vol_traded

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,Bridgefort Capital Limited,Bridgefort Capital Limited Class B,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-15 13:37:16.128091+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:38:57.928611+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:39:31.951745+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:40:51.702756+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:41:12.395476+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:41:49.528920+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:43:25.524984+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:44:00.977875+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0
2024-10-15 13:45:38.934554+02:00,0,0,4900,0,0,700,0,100,0,17200,...,74200,4200,0,2200,0,3000,0,0,100,0


In [25]:
schedule.every().day.do(update_data)

while True:
   schedule.run_pending()
   time.sleep(1)

   print('jobs:', len(schedule.get_jobs()))

jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3
jobs: 3


/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                      700   
2024-10-15 13:38:50.545881+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-15 13:37:04.878782+02:00                                              55.25    
2024-10-15 13:38:50.545881+02:00                                              55.25    

Company Name                     Ariston Holdings Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                        8   
2024-10-15 13:38:50.545881+02:00                        8   

Company Name                     Bridgefort Capital Limited  \
Date                                                          
2024-10-15 13:37:04.878782+02:0

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                      700   
2024-10-15 13:38:50.545881+02:00                      700   
2024-10-15 13:39:23.879831+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-15 13:37:04.878782+02:00                                              55.25    
2024-10-15 13:38:50.545881+02:00                                              55.25    
2024-10-15 13:39:23.879831+02:00                                              55.25    

Company Name                     Ariston Holdings Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                        8   
2024-10-15 13:38:50.545881+02:00                        8   
2024-10-1

...

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                      700   
2024-10-15 13:38:50.545881+02:00                      700   
2024-10-15 13:39:23.879831+02:00                      700   
2024-10-15 13:40:43.648451+02:00                      700   
2024-10-15 13:41:03.965778+02:00                      700   
2024-10-15 13:41:37.997257+02:00                      700   
2024-10-15 13:43:17.261272+02:00                      700   
2024-10-15 13:43:52.866361+02:00                      700   
2024-10-15 13:45:31.056505+02:00                      700   
2024-10-15 13:45:56.159749+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-15 13:37:04.878782+02:00                                              55.25    
202

/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)
/var/folders/7f/3tsf9r4d1sqd62b89lyh5cj40000gq/T/ipykernel_19122/4230871642.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(response.text)


Company Name                     Afdis Distillers Limited  \
Date                                                        
2024-10-15 13:37:04.878782+02:00                      700   
2024-10-15 13:38:50.545881+02:00                      700   
2024-10-15 13:39:23.879831+02:00                      700   
2024-10-15 13:40:43.648451+02:00                      700   
2024-10-15 13:41:03.965778+02:00                      700   
2024-10-15 13:41:37.997257+02:00                      700   
2024-10-15 13:43:17.261272+02:00                      700   
2024-10-15 13:43:52.866361+02:00                      700   
2024-10-15 13:45:31.056505+02:00                      700   
2024-10-15 13:45:56.159749+02:00                      700   
2024-10-15 13:46:11.399551+02:00                      700   

Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
Date                                                                                   
2024-10-15 13:37:04.878782+02:

KeyboardInterrupt: 

In [ ]:
open_price

In [ ]:
close_price

In [ ]:
vol_traded